In [1]:
import shapely as sh
import shapely
import phidl.geometry as pg
from phidl.path import Path,CrossSection
import matplotlib.pyplot as plt
%matplotlib qt
from phidl.quickplotter import quickplot2 as qp
import numpy as np
import utilities as util

In [2]:
alpha = 20
rad = 50
l=50
n_loops = 5
side_leg_length = 75
serpentine_width=74
outer_ring_rad=5000
outer_ring_width=1000
taper_start= serpentine_width
taper_end = 780
elec_radius=10

def serpentine(alpha,rad,l,n_loops,side_leg_length,buffer_radius=10,buffer_out_angle_radius=80):
    alpha =  alpha * np.pi/ 180
    t = np.linspace(-alpha,np.pi+alpha,100)[::-1]

    x,y = np.cos(t)*rad,np.sin(t)*rad
    y=y-np.min(y)+l/2*np.cos(alpha)

    l_coords = np.array([[x[-1],y[-1]],[x[-1]-l*np.sin(alpha),y[-1]-l*np.cos(alpha)]])

    x=np.hstack((x,l_coords[:,0]))
    y=np.hstack((y,l_coords[:,1]))

    shift = 2*rad - 2*rad*(1-np.cos(alpha)) - l*np.sin(alpha)

    newx,newy= x + 2*rad - 2*rad*(1-np.cos(alpha)) - l*np.sin(alpha), -y

    all_x,all_y=[],[]

    for i in range(5):
        plt.plot(x+i*shift,y*(-1)**(i-1))
        all_x.append(x+i*shift)
        all_y.append(y*(-1)**(i-1))
    all_x,all_y =np.concatenate(all_x)[:-1],np.concatenate(all_y)[:-1]

    all_x = np.hstack((all_x[0]+l/2*np.sin(alpha),all_x,all_x[-1]-l/2*np.sin(alpha)))
    all_x = np.hstack((all_x[0]-side_leg_length,all_x,all_x[-1]+side_leg_length))
    all_y = np.hstack((0,0,all_y,0,0))
    all_x-=all_x[0]
    span_serpentine = all_x[-1]-all_x[0]

    serp_string = shapely.LineString(  np.vstack((all_x,all_y)).T)

    closing_string = sh.LineString([sh.Point(all_x[0],0), sh.Point(all_x[0],3*rad), sh.Point(all_x[-1],3*rad), sh.Point(all_x[-1],0)])
    rounded_serpentine=sh.Polygon(sh.line_merge(closing_string.union(serp_string))).buffer(buffer_out_angle_radius).buffer(-buffer_out_angle_radius).exterior.difference(closing_string.buffer(.1))

    # ph_path=Path(rounded_serpentine.coords)
    # X=CrossSection()
    # X.add(width=5)
    # qp(ph_path.extrude(X))
    return rounded_serpentine.buffer(buffer_radius),span_serpentine,rounded_serpentine

base_pol,span,rounded_serpentine=serpentine(alpha=10,rad=110,l=70,n_loops=5,side_leg_length=350,buffer_radius=serpentine_width/2.,buffer_out_angle_radius=55)

In [3]:
center=sh.Point(0,0).buffer(150)
n_arms=8
serpentines=[]
for i in range(n_arms):
    angle = 360/n_arms*i
    serpentines.append(sh.affinity.rotate(base_pol,angle=angle,origin=(0,0)))
    electrodes_supports=sh.unary_union(rounded_serpentine.interpolate([0.19,0.5,.82],normalized=True))
    serpentines.append( sh.affinity.rotate(electrodes_supports.buffer(80),angle=angle,origin=(0,0)))
    
dev=util.shapely_to_phidl(sh.unary_union(serpentines).union(center))

for i in range(n_arms):
    angle = 360/n_arms*i
    taper = pg.taper(length=outer_ring_rad-outer_ring_width/2-taper_end-span,
                    width1=taper_start,width2=taper_end).movex(span).rotate(angle)
    square = pg.rectangle( (taper_end,taper_end)).movey(-taper_end/2.).movex(outer_ring_rad-outer_ring_width/2.-taper_end).rotate(angle)
    dev<<taper
    dev<<square
qp(dev)
dev<<pg.ring(radius=outer_ring_rad,width=outer_ring_width)
qp(dev)

In [4]:
sh_dev = util.phidl_to_shapely(dev).buffer(1).buffer(1)
router = util.RoutingLayout(sh_dev,border_tolerance=5)

In [5]:
router.lay_down_lines(n_lines=15,gap_um=5)

In [6]:
elec_center_mtp=sh.unary_union(rounded_serpentine.interpolate([0.19,0.5,.82],normalized=True))
top_xtr_pts = sh.affinity.translate(elec_center_mtp,yoff=80)
bot_xtr_pts = sh.affinity.translate(elec_center_mtp,yoff=-80)

elec_boundaries=[]
elec_extra_pts = []
elec_boundaries.append(sh.Point(0,0).buffer(50).boundary)
elec_extra_pts.append(sh.MultiPoint([sh.Point(-10,0),sh.Point(10,0)]))
for i in range(n_arms):
    angle = 360/n_arms*i
    for bdr,t,b in zip(elec_center_mtp.geoms,top_xtr_pts.geoms,bot_xtr_pts.geoms):
        elec_boundaries.append(sh.affinity.rotate( bdr.buffer(elec_radius).boundary, angle=angle, origin=(0,0)))
        elec_extra_pts.append( sh.affinity.rotate( sh.MultiPoint([t,b]), angle=angle, origin=(0,0)))

router.add_electrodes(elec_boundaries,elec_extra_pts,type='electrode')


In [7]:
qp(util.multilinestrings_to_phidl(router.all_route_lines))

In [8]:
pad_boundaries=[]
pad_extra_pts=[]
for i in range(len(elec_boundaries)):
    angle = ( 270 - len(elec_boundaries)//2 + i ) * np.pi/180
    r=outer_ring_rad+outer_ring_width//2 
    x,y= r*np.cos(angle),r*np.sin(angle)
    xpt,ypt=(r-outer_ring_width//2*1.2)*np.cos(angle), (r-outer_ring_width//2*1.2)*np.sin(angle)
    pad_boundaries.append( sh.Point(x,y).buffer(10).boundary)
    pad_extra_pts.append( sh.MultiPoint([sh.Point(xpt,ypt)]))

In [9]:
router.add_electrodes(pad_boundaries,pad_extra_pts,type='pad')


In [10]:
qp(util.multilinestrings_to_phidl(router.all_route_lines))

In [11]:
router.route()

Routing successful


In [12]:
full_dev=pg.Device()

In [13]:
full_dev<<dev
qp(full_dev)
elec_lines=util.multilinestrings_to_phidl(sh.unary_union(router.all_elec_pad_routes_smooth),width=2,layer=1)
full_dev<<elec_lines


DeviceReference (parent Device "Unnamed", ports [], origin (0, 0), rotation 0, x_reflection False)

In [14]:
qp(full_dev)